In [ ]:
import os
import customtkinter as ctk
from tkshadowfy import Shadow
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tkinter import messagebox

# Setup dark theme
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("dark-blue")

# Auto-detect dataset files
data_folder = "C:/Users/DELL/Downloads/tmdb_5000_movies"
movies_file, credits_file = None, None
for f in os.listdir(data_folder):
    if "movie" in f.lower() and f.endswith(".csv"):
        movies_file = os.path.join(data_folder, f)
    if "credit" in f.lower() and f.endswith(".csv"):
        credits_file = os.path.join(data_folder, f)
if not movies_file or not credits_file:
    raise FileNotFoundError("❌ Could not find TMDB dataset CSVs in the folder.")

print("Using:", movies_file, credits_file)

# Load and merge datasets
movies = pd.read_csv(movies_file)
credits = pd.read_csv(credits_file)
movies = movies.merge(credits, left_on='id', right_on='movie_id')
movies = movies[['id','title_x','overview','genres','cast','crew']].rename(columns={'title_x':'title'})

# Add 20 Bollywood movies manually
bollywood_list = [
    {'id':10001, 'title':'3 Idiots', 'overview':"Two friends search for their long‑lost companion...", 'genres':"Comedy Drama", 'cast':"Aamir Khan R. Madhavan Sharman Joshi Kareena Kapoor", 'crew':"Rajkumar Hirani"},
    {'id':10002, 'title':'Sholay', 'overview':"Two criminals are hired by a retired police officer...", 'genres':"Action Adventure Drama", 'cast':"Amitabh Bachchan Dharmendra Hema Malini Amjad Khan", 'crew':"Ramesh Sippy"},
    {'id':10003, 'title':'Dilwale Dulhania Le Jayenge', 'overview':"Raj and Simran fall in love during a trip...", 'genres':"Romance Drama", 'cast':"Shah Rukh Khan Kajol Amrish Puri", 'crew':"Aditya Chopra"},
    {'id':10004, 'title':'Kabhi Khushi Kabhie Gham', 'overview':"An ambitious young man returns home for love...", 'genres':"Drama Family Romance", 'cast':"Shah Rukh Khan Kajol Amitabh Bachchan Hrithik Roshan", 'crew':"Karan Johar"},
    {'id':10005, 'title':'PK', 'overview':"An alien lands on Earth and questions religious dogmas...", 'genres':"Comedy Drama Fantasy", 'cast':"Aamir Khan Anushka Sharma Sushant Singh Rajput", 'crew':"Rajkumar Hirani"},
    {'id':10006, 'title':'Chak De! India', 'overview':"A disgraced hockey player coaches the Indian women's team...", 'genres':"Drama Sport", 'cast':"Shah Rukh Khan Vidya Malvade Sagarika Ghatge", 'crew':"Shimit Amin"},
    {'id':10007, 'title':'Taare Zameen Par', 'overview':"A dyslexic child discovers his love for art...", 'genres':"Drama Family", 'cast':"Darsheel Safary Aamir Khan Tisca Chopra", 'crew':"Aamir Khan"},
    {'id':10008, 'title':'Lagaan', 'overview':"Villagers challenge British officers to a cricket match...", 'genres':"Drama Sport History", 'cast':"Aamir Khan Gracy Singh Rachel Shelley", 'crew':"Ashutosh Gowariker"},
    {'id':10009, 'title':'Devdas', 'overview':"A tragic love story of a man who returns to India...", 'genres':"Drama Romance", 'cast':"Shah Rukh Khan Aishwarya Rai Madhuri Dixit", 'crew':"Sanjay Leela Bhansali"},
    {'id':10010, 'title':'Mughal-e-Azam', 'overview':"A historic romance between a prince and a court dancer...", 'genres':"Drama History Romance", 'cast':"Prithviraj Kapoor Madhubala Dilip Kumar", 'crew':"K. Asif"},
    {'id':10011, 'title':'Gully Boy', 'overview':"An aspiring street rapper from Mumbai emerges...", 'genres':"Drama Music", 'cast':"Ranveer Singh Alia Bhatt Vijay Raaz", 'crew':"Zoya Akhtar"},
    {'id':10012, 'title':'Barfi', 'overview':"Three hearts collide—two physically challenged...", 'genres':"Comedy Drama Romance", 'cast':"Ranbir Kapoor Priyanka Chopra Ileana D'Cruz", 'crew':"Anurag Basu"},
    {'id':10013, 'title':'Queen', 'overview':"A young bride goes on her honeymoon alone and discovers herself.", 'genres':"Comedy Drama Romance", 'cast':"Kangana Ranaut Rajkummar Rao Lisa Haydon", 'crew':"Vikas Bahl"},
    {'id':10014, 'title':'Dangal', 'overview':"A former wrestler trains his daughters to become world‑class wrestlers.", 'genres':"Drama Sport Biography", 'cast':"Aamir Khan Fatima Sana Shaikh Sanya Malhotra", 'crew':"Nitesh Tiwari"},
    {'id':10015, 'title':'Bajrangi Bhaijaan', 'overview':"A man helps a mute girl return to her home in Pakistan.", 'genres':"Adventure Comedy Drama", 'cast':"Salman Khan Kareena Kapoor Khan Nawazuddin Siddiqui", 'crew':"Kabir Khan"},
    {'id':10016, 'title':'Zindagi Na Milegi Dobara', 'overview':"Three friends go on a road trip that changes their lives.", 'genres':"Comedy Drama Romance", 'cast':"Hrithik Roshan Farhan Akhtar Abhay Deol Katrina Kaif", 'crew':"Zoya Akhtar"},
    {'id':10017, 'title':'Rang De Basanti', 'overview':"A group of young friends bring change after a tragedy.", 'genres':"Drama History", 'cast':"Aamir Khan Siddharth Narayan Sharman Joshi Soha Ali Khan", 'crew':"Rakeysh Omprakash Mehra"},
    {'id':10018, 'title':'Kal Ho Naa Ho', 'overview':"A terminally ill man teaches love to his friend.", 'genres':"Drama Romance Comedy", 'cast':"Shah Rukh Khan Preity Zinta Saif Ali Khan", 'crew':"Nikhil Advani"},
    {'id':10019, 'title':'Kuch Kuch Hota Hai', 'overview':"A love story between friends and college romance.", 'genres':"Romance Drama", 'cast':"Shah Rukh Khan Kajol Rani Mukerji", 'crew':"Karan Johar"},
    {'id':10020, 'title':'Andaz Apna Apna', 'overview':"Two slackers compete for a woman’s attention, causing chaos.", 'genres':"Comedy", 'cast':"Aamir Khan Salman Khan Raveena Tandon", 'crew':"Rajkumar Santoshi"}
]

movies = pd.concat([movies, pd.DataFrame(bollywood_list)], ignore_index=True)
for col in ['overview', 'genres', 'cast', 'crew']:
    movies[col] = movies[col].fillna('')
movies['tags'] = movies['overview'] + ' ' + movies['genres'] + ' ' + movies['cast'] + ' ' + movies['crew']
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(movies['tags']).toarray()
similarity = cosine_similarity(vectors)

def recommend(name):
    if name not in movies['title'].values:
        return []
    idx = movies[movies['title'] == name].index[0]
    dists = similarity[idx]
    return [movies.iloc[i[0]].title for i in sorted(list(enumerate(dists)), reverse=True, key=lambda x: x[1])[1:6]]

# ------------------------ MODERN GUI START ------------------------

root = ctk.CTk()
root.title("Smart Bollywood & Hollywood Recommender")
root.geometry("800x650")
root.configure(bg_color="#0f0f0f")  # darker background

# Moving marquee text
marquee_text = "🎬 Welcome to the Smart Bollywood & Hollywood Movie Recommender! 🎬  "
marquee_label = ctk.CTkLabel(root, text=marquee_text, font=("Segoe UI", 14, "bold"), text_color="#00FFAA", bg_color="#0f0f0f")
marquee_label.pack(pady=10)

def scroll_text():
    current = marquee_label.cget("text")
    marquee_label.configure(text=current[1:] + current[0])
    root.after(150, scroll_text)

scroll_text()

# Header
header_frame = ctk.CTkFrame(root, fg_color="#1a1a1a", corner_radius=10)
header_frame.pack(pady=10, padx=20, fill="x")

header = ctk.CTkLabel(header_frame, text="Smart Movie Recommender", text_color="#e50914", font=("Segoe UI", 22, "bold"))
header.pack(pady=10)
Shadow(header, color="#e50914", size=10, offset_x=0, offset_y=0)

# Search section
search_frame = ctk.CTkFrame(root, fg_color="#1a1a1a", corner_radius=10)
search_frame.pack(pady=10, padx=20, fill="x")

search_label = ctk.CTkLabel(search_frame, text="Enter Movie Title:", text_color="white", font=("Segoe UI", 14))
search_label.pack(pady=(10, 5))

search_var = ctk.StringVar()
search_entry = ctk.CTkEntry(search_frame, textvariable=search_var, width=500, height=40, fg_color="#2b2b2b", text_color="white", placeholder_text="e.g. 3 Idiots")
search_entry.pack(pady=(0, 10))

# Button
button = ctk.CTkButton(search_frame, text="🎥 Get Recommendations", command=lambda: show_recs(), fg_color="#e50914", hover_color="#f6121d", font=("Segoe UI", 14, "bold"))
button.pack(pady=(0, 10))

# Recommendation box
rec_frame = ctk.CTkFrame(root, fg_color="#1a1a1a", corner_radius=10)
rec_frame.pack(pady=20, padx=20, fill="both", expand=True)

rec_label = ctk.CTkLabel(rec_frame, text="Recommended Movies:", font=("Segoe UI", 16, "bold"), text_color="#00FFAA")
rec_label.pack(pady=10)

rec_listbox = ctk.CTkTextbox(rec_frame, width=600, height=200, fg_color="#121212", text_color="white", font=("Segoe UI", 12))
rec_listbox.pack(pady=10)

# Footer
footer = ctk.CTkLabel(root, text="— Created by Umme Kulsum", text_color="#555555", font=("Segoe UI", 10))
footer.pack(side="bottom", pady=10)

# Show recommendations
def show_recs():
    movie = search_var.get().strip()
    recs = recommend(movie)
    rec_listbox.delete("1.0", "end")
    if recs:
        for i, r in enumerate(recs, 1):
            rec_listbox.insert("end", f"{i}. {r}\n")
    else:
        rec_listbox.insert("end", f"❌ Sorry, '{movie}' not found in database.\nTry another title.")

root.mainloop()

Using: C:/Users/DELL/Downloads/tmdb_5000_movies\tmdb_5000_movies.csv C:/Users/DELL/Downloads/tmdb_5000_movies\tmdb_5000_credits.csv
